## Implementation of RBF networks and test on adversarial examples

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from keras.layers import Layer
from keras import backend as K


C:\Users\Bounadja Bilal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## We load MNIST dataset

In [4]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## We generate adversarial examples using the fast gradient sign method

In [5]:
'''def get_adversial_exemple(model, dataset, eps):
  adversarial_examples = []
  for images, labels in dataset:
    with tf.GradientTape() as tape:
      tape.watch(images)
      predictions = model(images)
      labels = tf.expand_dims(labels, 1)
      
      loss = tf.keras.losses.binary_crossentropy(labels, predictions)
    gradient = tape.gradient(loss, images)
    signed_grad = tf.sign(gradient)
    adversarial_example = images + eps * signed_grad
    adversarial_example = tf.clip_by_value(adversarial_example, 0, 1)
    adversarial_examples.append(adversarial_example)
  return np.concatenate(adversarial_examples,axis=0)'''
  
  
def get_adversial_exemple(model, image, label, eps=0.25):
    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image, 0)
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.sparse_categorical_crossentropy(label, prediction)
    gradient = tape.gradient(loss, image)
    signed_grad = tf.sign(gradient)
    adv_ex = image + eps*signed_grad    
    adv_ex = tf.clip_by_value(adv_ex, 0, 1)
    return adv_ex

## RBF Layer definition
Simple definition of an RBF layer

In [6]:



class RBFLayer(Layer):
    def __init__(self, units, beta, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.beta = K.cast_to_floatx(beta)

    def build(self, input_shape):
#        print(input_shape)
#         print(self.units)
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu #x-mu
        l2 = K.sum(K.pow(diff, 2), axis=1) #somme des x-mu au carré
        return K.exp(-1 * self.beta * l2) #multiplication par beta

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

## Implementation of Shallow RBF Network

In [7]:

from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential
from keras.losses import binary_crossentropy
RBF_Shallow = Sequential()
RBF_Shallow.add(Flatten(input_shape=(28, 28)))
RBF_Shallow.add(RBFLayer(1000, 0.25))

RBF_Shallow.add(Dense(10,activation='sigmoid'))

RBF_Shallow.compile(optimizer='rmsprop', loss="SparseCategoricalCrossentropy",metrics=['accuracy'])


We can load a pre trained model

In [ ]:
#RBF_Shallow = tf.keras.models.load_model ('RBFvanilla.h5') 

Training

In [41]:
RBF_Shallow.fit(ds_train, batch_size=256, epochs=1)
RBF_Shallow.evaluate(ds_test)

79/79 [==============================] - 9s 110ms/step - loss: 0.8270 - accuracy: 0.7492


[0.826958179473877, 0.7491999864578247]

We can save the model

In [42]:
#tf.saved_model.save(RBF_Shallow, r'C:\Users\Bounadja Bilal\Documents\GitHub\ProjetMLA')

INFO:tensorflow:Assets written to: C:\Users\Bounadja Bilal\Documents\GitHub\ProjetMLA\assets


INFO:tensorflow:Assets written to: C:\Users\Bounadja Bilal\Documents\GitHub\ProjetMLA\assets


In [43]:

# convert tf.data.Dataset to numpy array
x_test = np.concatenate([x for x, y in ds_test], axis=0)
y_test = np.concatenate([y for x, y in ds_test], axis=0)


## Evaluation on Adversarial Examples

In [44]:
'''x_test_adv = get_adversial_exemple(RBF_Shallow, ds_test, eps=0.25)
loss, accuracy =  RBF_Shallow.evaluate(x_test_adv, y_test)
confidence = np.mean(np.max(RBF_Shallow.predict(x_test_adv), axis=1))
print(f"Accuracy on adversarial examples: {accuracy} with a mean confidence of {confidence}", )'''
x_test_adv = [get_adversial_exemple(RBF_Shallow, x, y,eps=0.25).numpy()[0] for x,y in zip(x_test, y_test)]
x_test_adv = np.array(x_test_adv)

l,a = RBF_Shallow.evaluate(x_test_adv, y_test)
c = np.mean(np.max(RBF_Shallow.predict(x_test_adv), axis=1))
print(f"Accuracy on adversarial examples: {a} with a mean confidence of {c}", )

313/313 [==============================] - 10s 33ms/step
Accuracy on adversarial examples: 0.09790000319480896 with a mean confidence of 0.0971347838640213
